In [19]:
import pandas as pd
import json
import random
from datetime import datetime, timedelta
from allpairspy import AllPairs
import numpy as np

In [1]:


# Define the path to your claims JSON file
claims_file_path = 'synthetic_facets_claims_1000.json'

# Load the JSON data
with open(claims_file_path, 'r') as f:
    claims_data = json.load(f)

# Normalize the 'CLAIM_HEADER' part
# We treat each element in the claims_data list as a record.
# The 'CLAIM_HEADER' is a nested dictionary that we want to flatten.
# The 'CLAIM_LINE_ITEMS' is a list within each record that needs separate normalization.
df_claims_header = pd.json_normalize(
    claims_data,
    record_path=None, # No specific record_path here, as the main objects are the claims themselves
    meta=['CLAIM_HEADER'] # We want to pull fields from CLAIM_HEADER
)

# Rename columns to flatten the 'CLAIM_HEADER.' prefix
df_claims_header.columns = [col.replace('CLAIM_HEADER.', '') for col in df_claims_header.columns]

# Now, normalize the 'CLAIM_LINE_ITEMS'
# This requires iterating through each claim and its line items
all_line_items = []
for claim in claims_data:
    claim_id = claim['CLAIM_HEADER']['CLAIM_ID']
    for line_item in claim['CLAIM_LINE_ITEMS']:
        # Add the CLAIM_ID from the header to each line item for linking
        line_item['CLAIM_ID'] = claim_id
        all_line_items.append(line_item)

df_claims_lines = pd.DataFrame(all_line_items)

print("Claims Header DataFrame (first 5 rows):")
print(df_claims_header.head())
print("\nClaims Line Items DataFrame (first 5 rows):")
print(df_claims_lines.head())

print(f"\nClaims Header DataFrame shape: {df_claims_header.shape}")
print(f"Claims Line Items DataFrame shape: {df_claims_lines.shape}")

Claims Header DataFrame (first 5 rows):
                                    CLAIM_LINE_ITEMS   CLAIM_ID MEMBER_ID  \
0  [{'CLAIM_ID': 'CLM000001', 'LINE_NUMBER': 1, '...  CLM000001   MEM0163   
1  [{'CLAIM_ID': 'CLM000002', 'LINE_NUMBER': 1, '...  CLM000002   MEM0198   
2  [{'CLAIM_ID': 'CLM000003', 'LINE_NUMBER': 1, '...  CLM000003   MEM0029   
3  [{'CLAIM_ID': 'CLM000004', 'LINE_NUMBER': 1, '...  CLM000004   MEM0119   
4  [{'CLAIM_ID': 'CLM000005', 'LINE_NUMBER': 1, '...  CLM000005   MEM0190   

  PROVIDER_ID  PLAN_ID SERVICE_START_DATE SERVICE_END_DATE  \
0     PROV009  PLAN003         2024-07-31       2024-07-31   
1     PROV003  PLAN003         2024-12-16       2024-12-16   
2     PROV001  PLAN002         2024-10-08       2024-10-08   
3     PROV018  PLAN001         2024-12-30       2024-12-30   
4     PROV029  PLAN001         2023-11-06       2023-11-06   

  CLAIM_SUBMISSION_DATE DIAGNOSIS_CODE_PRIMARY CLAIM_STATUS  \
0            2024-08-28                    R51       Denied  

In [2]:
# Define the path to your members JSON file
members_file_path = 'synthetic_members.json'

# Read the JSON file directly into a DataFrame
df_members = pd.read_json(members_file_path)

print("\nMembers DataFrame (first 5 rows):")
print(df_members.head())
print(f"\nMembers DataFrame shape: {df_members.shape}")


Members DataFrame (first 5 rows):
  MEMBER_ID FIRST_NAME LAST_NAME DATE_OF_BIRTH GENDER  \
0   MEM0001    Matthew   Spencer    1991-08-14      F   
1   MEM0002      Robin     Gates    1980-02-29      M   
2   MEM0003    William     Lopez    2006-03-22      F   
3   MEM0004    Douglas      Long    1946-01-14      M   
4   MEM0005     Joseph    Taylor    2011-04-13      F   

                                             ADDRESS              CITY STATE  \
0                   Unit 5688 Box 2344, DPO AA 75012  East Taylorhaven    MO   
1  93706 Tyler Plains Suite 731, West Christopher...         Clarkstad    VT   
2  6417 Zamora Crossing Suite 302, Dylanborough, ...    North Kimberly    MT   
3  925 Sabrina Extensions Apt. 630, Penningtonbor...       Jefferytown    FM   
4  04396 Angel Walks Apt. 441, East Matthew, IN 9...         Eatonland    MA   

   ZIP_CODE  
0     75432  
1      6803  
2     87637  
3     63897  
4     74767  

Members DataFrame shape: (200, 9)


In [3]:
# Define the path to your providers JSON file
providers_file_path = 'synthetic_providers.json'

# Read the JSON file directly into a DataFrame
df_providers = pd.read_json(providers_file_path)

print("\nProviders DataFrame (first 5 rows):")
print(df_providers.head())
print(f"\nProviders DataFrame shape: {df_providers.shape}")


Providers DataFrame (first 5 rows):
  PROVIDER_ID                NAME        SPECIALTY         NPI  \
0     PROV001     Terry Garcia MD       Cardiology  4621735579   
1     PROV002  Jennifer Turner MD      Orthopedics  6790835900   
2     PROV003  Zachary Spencer DO       Cardiology  1781446297   
3     PROV004      Jimenez-Peters  Family Medicine  3119503147   
4     PROV005      Karen Smith MD      Orthopedics  8906101802   

                                             ADDRESS              CITY STATE  \
0  9804 Freeman Walks Apt. 570, South Sherrychest...    Port Elizabeth    ID   
1  5018 Janet Mount Suite 858, East Jenniferside,...        Smithmouth    HI   
2  869 Christine Mills Apt. 373, Lake Stephaniemo...      Hollowaybury    AK   
3  700 Gentry Tunnel Suite 702, Torresbury, NE 24141       North Jamie    NV   
4         71009 Sarah Junction, Thomasfort, NH 29990  New Cindychester    MS   

   ZIP_CODE  
0     92339  
1      7655  
2     56901  
3     91760  
4     18753  



In [4]:
# Define the path to your plans JSON file
plans_file_path = 'synthetic_plans.json'

# Read the JSON file directly into a DataFrame
df_plans = pd.read_json(plans_file_path)

print("\nPlans DataFrame (first 5 rows):")
print(df_plans.head())
print(f"\nPlans DataFrame shape: {df_plans.shape}")


Plans DataFrame (first 5 rows):
   PLAN_ID              NAME TYPE  COPAY  DEDUCTIBLE  COINSURANCE
0  PLAN001      BlueCare PPO  PPO     30         500         0.20
1  PLAN002     HealthNet HMO  HMO     15         100         0.10
2  PLAN003  SecureChoice EPO  EPO     25        1000         0.25

Plans DataFrame shape: (3, 6)


In [5]:
# Merge claims header with members
df_claims_enriched = pd.merge(
    df_claims_header,
    df_members,
    left_on='MEMBER_ID',
    right_on='MEMBER_ID',
    how='left', # Keep all claims, add member info where available
    suffixes=('_claim', '_member') # To distinguish columns with same names
)

# Merge with providers
df_claims_enriched = pd.merge(
    df_claims_enriched,
    df_providers,
    left_on='PROVIDER_ID',
    right_on='PROVIDER_ID',
    how='left',
    suffixes=('_claims', '_provider') # Another suffix for new merge
)

# Merge with plans
df_claims_enriched = pd.merge(
    df_claims_enriched,
    df_plans,
    left_on='PLAN_ID',
    right_on='PLAN_ID',
    how='left',
    suffixes=('_claims', '_plan')
)

print("\nEnriched Claims Header DataFrame (first 5 rows with merged data):")
print(df_claims_enriched.head())
print(f"\nEnriched Claims Header DataFrame shape: {df_claims_enriched.shape}")

# You would do similar merges if you want to link claim lines to members/providers/plans
# (though usually the header holds the main linkages)


Enriched Claims Header DataFrame (first 5 rows with merged data):
                                    CLAIM_LINE_ITEMS   CLAIM_ID MEMBER_ID  \
0  [{'CLAIM_ID': 'CLM000001', 'LINE_NUMBER': 1, '...  CLM000001   MEM0163   
1  [{'CLAIM_ID': 'CLM000002', 'LINE_NUMBER': 1, '...  CLM000002   MEM0198   
2  [{'CLAIM_ID': 'CLM000003', 'LINE_NUMBER': 1, '...  CLM000003   MEM0029   
3  [{'CLAIM_ID': 'CLM000004', 'LINE_NUMBER': 1, '...  CLM000004   MEM0119   
4  [{'CLAIM_ID': 'CLM000005', 'LINE_NUMBER': 1, '...  CLM000005   MEM0190   

  PROVIDER_ID  PLAN_ID SERVICE_START_DATE SERVICE_END_DATE  \
0     PROV009  PLAN003         2024-07-31       2024-07-31   
1     PROV003  PLAN003         2024-12-16       2024-12-16   
2     PROV001  PLAN002         2024-10-08       2024-10-08   
3     PROV018  PLAN001         2024-12-30       2024-12-30   
4     PROV029  PLAN001         2023-11-06       2023-11-06   

  CLAIM_SUBMISSION_DATE DIAGNOSIS_CODE_PRIMARY CLAIM_STATUS  ...         NPI  \
0            2024

In [6]:
df_claims_lines

,CLAIM_ID,LINE_NUMBER,SERVICE_START_DATE,SERVICE_END_DATE,PROCEDURE_CODE,MODIFIER,DIAGNOSIS_CODE_PRIMARY,UNITS,LINE_BILLED_AMOUNT,LINE_ALLOWED_AMOUNT,LINE_PAID_AMOUNT,LINE_MEMBER_COPAY,LINE_MEMBER_DEDUCTIBLE,LINE_MEMBER_COINSURANCE,PLACE_OF_SERVICE_CODE,REMARK_CODE_1
0,CLM000001,1,2024-07-31,2024-07-31,99214,25,R51,1,198.27,0.00,0.0,0.00,0.00,0.0,12,CO45
1,CLM000002,1,2024-12-16,2024-12-16,99204,None,N39.0,1,237.59,0.00,0.0,0.00,0.00,0.0,22,CO97
2,CLM000002,2,2024-12-16,2024-12-16,99213,59,N39.0,1,113.58,95.39,0.0,25.00,70.39,0.0,12,None
3,CLM000002,3,2024-12-16,2024-12-16,71046,59,N39.0,1,147.08,138.80,0.0,25.00,113.80,0.0,21,None
4,CLM000003,1,2024-10-08,2024-10-08,90837,None,E11.9,1,276.03,0.00,0.0,0.00,0.00,0.0,12,PR204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974,CLM000998,3,2024-08-09,2024-08-09,90837,25,Z00.00,1,223.33,186.32,0.0,25.00,161.32,0.0,11,None
1975,CLM000999,1,2023-02-10,2023-02-10,36415,59,I10,1,16.33,12.22,0.0,12.22,0.00,0.0,11,None
1976,CLM001000,1,2023-09-18,2023-09-18,99214,59,R51,1,196.72,170.07,0.0,30.00,140.07,0.0,12,None
1977,CLM001000,2,2023-09-18,2023-09-18,90672,25,R51,1,62.80,57.23,0.0,30.00,27.23,0.0,21,None


## **Reading and creating all pairs**

In [7]:
df_proc_diag = pd.read_excel("Diag_Proc_Code_AllPairs.xlsx")
df_proc_diag

,DiagCode,ProcCode
0,J06.9,99213
1,I10,99203
2,E11.9,80061
3,K21.9,90672
4,R51,71046
5,N39.0,90837
6,M54.5,81002
7,F41.9,99204
8,Z00.00,99214
9,NaN,36415


In [12]:
p = [df_proc_diag[col].dropna().tolist() for col in df_proc_diag.columns]
combinations = AllPairs(p)


In [13]:
df_combinations = pd.DataFrame(list(combinations), columns = df_proc_diag.columns)
df_combinations

,DiagCode,ProcCode
0,J06.9,99213
1,I10,99213
2,E11.9,99213
3,K21.9,99213
4,R51,99213
...,...,...
85,R51,36415
86,K21.9,36415
87,E11.9,36415
88,I10,36415


In [14]:
df_source_rates = pd.read_excel("Proc_code_Rate_Allpairs.xlsx")
df_source_dict = df_source_rates.set_index('ProcCode')['Allowed_Amount'].to_dict()
df_source_dict

{99213: 120,
 99203: 180,
 80061: 40,
 90672: 75,
 71046: 150,
 90837: 250,
 81002: 30,
 99204: 250,
 99214: 180,
 36415: 20}

In [15]:
df_combinations['Allowed_Amount'] = df_combinations['ProcCode'].apply(
    lambda x: df_source_dict.get(x, None) # Or 0, or pd.NA
)

In [16]:
df_combinations

,DiagCode,ProcCode,Allowed_Amount
0,J06.9,99213,120
1,I10,99213,120
2,E11.9,99213,120
3,K21.9,99213,120
4,R51,99213,120
...,...,...,...
85,R51,36415,20
86,K21.9,36415,20
87,E11.9,36415,20
88,I10,36415,20


In [20]:
def get_actual_amount(row, df_claims_lines):
    match = df_claims_lines[(df_claims_lines['DIAGNOSIS_CODE_PRIMARY'] == row['DiagCode']) & (df_claims_lines['PROCEDURE_CODE'] == row['ProcCode'])]
    return match['LINE_ALLOWED_AMOUNT'].iloc[0] if not match.empty else np.nan

In [21]:
df_combinations['actual_amount'] = df_combinations.apply(get_actual_amount, axis=1, args=(df_claims_lines,))
df_combinations

,DiagCode,ProcCode,Allowed_Amount,actual_amount
0,J06.9,99213,120,NaN
1,I10,99213,120,NaN
2,E11.9,99213,120,NaN
3,K21.9,99213,120,NaN
4,R51,99213,120,NaN
...,...,...,...,...
85,R51,36415,20,NaN
86,K21.9,36415,20,NaN
87,E11.9,36415,20,NaN
88,I10,36415,20,NaN


In [22]:
df_claims_lines.columns

Index(['CLAIM_ID', 'LINE_NUMBER', 'SERVICE_START_DATE', 'SERVICE_END_DATE',
       'PROCEDURE_CODE', 'MODIFIER', 'DIAGNOSIS_CODE_PRIMARY', 'UNITS',
       'LINE_BILLED_AMOUNT', 'LINE_ALLOWED_AMOUNT', 'LINE_PAID_AMOUNT',
       'LINE_MEMBER_COPAY', 'LINE_MEMBER_DEDUCTIBLE',
       'LINE_MEMBER_COINSURANCE', 'PLACE_OF_SERVICE_CODE', 'REMARK_CODE_1'],
      dtype='object')